In [30]:
from copy import deepcopy


def find_in_vertex(n):
    v = 2 * abs(n) - 2
    if n < 0:
        v += 1
    return v


def find_out_vertex(n):
    v = 2 * abs(n) - 1
    if n < 0:
        v -= 1
    return v


class PermutationGraph:
    def __init__(self, str):
        cycles = [list(map(int, s[:-1].split())) for s in str.split('(')]
        self._n = sum(len(cycle) for cycle in cycles)
        self._graph = [[] for _ in range(2 * self._n)]
        for cycle in cycles:
            self._add_egdes(cycle, self._graph)

    @staticmethod
    def _add_egdes(cycle, graph):
        n = len(cycle)
        for i in range(n):
            j = (i + 1) % n
            in_vertex = find_in_vertex(cycle[j])
            out_vertex = find_out_vertex(cycle[i])
            graph[out_vertex].append(in_vertex)
            graph[in_vertex].append(out_vertex)

    @staticmethod
    def _signed_str(n):
        if n > 0:
            return '+{}'.format(n)
        return str(n)

    def _traverse_cycle(self, v, visited, cycle_nodes):
        visited[v] = 1
        if v % 2 == 0:
            cycle_nodes.append(v // 2 + 1)
        else:
            cycle_nodes.append(-(v // 2) - 1)
        next_v = v - 1 if v % 2 else v + 1
        if visited[next_v]:
            return
        visited[next_v] = 1
        next_v = self._graph[next_v][0]
        if visited[next_v]:
            return
        self._traverse_cycle(next_v, visited, cycle_nodes)

    def __str__(self):
        visited = [0 for _ in range(2 * self._n)]
        permutation = []
        for v in range(2 * self._n):
            if visited[v]:
                continue
            cycle = []
            self._traverse_cycle(v, visited, cycle)
            permutation.append(cycle)
        return ' '.join([
            '({})'.format(' '.join(map(self._signed_str, cycle)))
            for cycle in permutation
            ])

    def __eq__(self, other):
        return list(map(sorted, self._graph)) == \
                list(map(sorted, other._graph))

    def __hash__(self):
        return hash(str(self))

    def two_break(self, u, v, x, y):
        """conduct 2-break between edges (u,v) and (x,y)
        replace edges with (u,x) and (v,y)
        """
        for from_v, to_v in [(u, v), (v, u), (x, y), (y, x)]:
            self._graph[from_v].remove(to_v)
        for from_v, to_v in [(u, x), (x, u), (v, y), (y, v)]:
            self._graph[from_v].append(to_v)

    def graph(self):
        return deepcopy(self._graph)


def main():
    
    file = open('rosalind_ba6k.txt', 'r')

    p = PermutationGraph(next(file).strip())
    u, v, x, y = map(int, next(file).strip().split(', '))
    p.two_break(u - 1, v - 1, x - 1, y - 1)
    print(p)

In [31]:
if __name__ == '__main__':
    main()

(+1 +70 +69 -68 +67 +66 -65 -64 -63 +62 -61 -60 +59 -58 -57 +56 +55 +54 +53 -52 -51 -50 -49 +48 -47 -46 +45 -44 +43 -42 -41 -40 -39 -38 +37 -36 -35 -34 -33 -32 +31 -30 -29 +28 -27 +26 +25 -24 -23 -22 +21 +20 +19 +18 +17 -16 -15 -14 -13 +12 +11 -10 -9 -8 -7 +6 +5 -4 -3 +2)


In [23]:
see = '(+1 -2 -4 +3)'

cycles = [list(map(int, s[:-1].split())) for s in see.split('(')]

In [16]:
cycles = [list(map(int, s[:-1].split())) for s in see.split('(')]
cycles

[[], [1, -2, -4, 3]]

In [29]:
seq = see
seq = seq[1:-1].split()
seq = list(map(int, seq))
seq

[1, -2, -4, 3]

['+1', '-2', '-4', '+3']

In [28]:
list(map(int, see[1:-1].split()))

[1, -2, -4, 3]

In [32]:
from collections import defaultdict
from copy import deepcopy
import numpy as np


def get_u_in(n):
    v = 2 * abs(n) - 2
    if n < 0:
        v += 1
    return v


def get_v_out(n):
    v = 2 * abs(n) - 1
    if n < 0:
        v -= 1
    return v


class Graph:
    def __init__(self):
        self.Adj = defaultdict(list)

    def build_graph(self, cycles):
        for cycle in cycles:
            self.create_edge(cycle)

    def create_edge(self, cycle,):
        n = len(cycle)
        for i in range(n):
            j = (i + 1) % n
            u = get_u_in(cycle[j])
            v = get_v_out(cycle[i])
            self.Adj[v].append(u)
            self.Adj[u].append(v)

    def get_two_break_on_graph(self, i1, i2, j1, j2):
        for u, v in [(i1, i2), (i2, i1), (j1, j2), (j2, j1)]:
            self.Adj[u].remove(v)
        for u, v in [(i1, j1), (j1, i1), (i2, j2), (j2, i2)]:
            self.Adj[u].append(v)

    def dfs_visit(self, u, visited, nodes_in_cycle):
        visited.add(u)
        if u % 2 == 0:
            nodes_in_cycle.append(u // 2 + 1)
        else:
            nodes_in_cycle.append(-(u // 2) - 1)
        v = u - 1 if u % 2 else u + 1
        if v not in visited:
            visited.add(v)
            v = self.Adj[v][0]
            if v in visited:
                return
            self.dfs_visit(v, visited, nodes_in_cycle)

    def graph_to_genome(self, n):
        visited = set()
        resulting_genome = []
        for v in range(2 * n):
            if v not in visited:
                nodes_in_cycle = []
                self.dfs_visit(v, visited, nodes_in_cycle)
                resulting_genome.append(nodes_in_cycle)

        result = []
        for nodes_in_cycle in resulting_genome:
            res = [str(u) if u < 0 else '+' + str(u) for u in nodes_in_cycle]
            result.append('(' + ' '.join(res) + ')')

        return ' '.join(result)


def main():

    file = open('rosalind_ba6k.txt', 'r')

    genome = next(file).strip()
    indices = np.array(list(map(int, next(file).strip().split(', ')))) - 1

    cycles = [list(map(int, s[:-1].split())) for s in genome.split('(')]
    n = sum(len(cycle) for cycle in cycles)

    graph = Graph()
    graph.build_graph(cycles)
    graph.get_two_break_on_graph(*indices)
    print(graph.graph_to_genome(n))


if __name__ == '__main__':
    main()

(+1 +70 +69 -68 +67 +66 -65 -64 -63 +62 -61 -60 +59 -58 -57 +56 +55 +54 +53 -52 -51 -50 -49 +48 -47 -46 +45 -44 +43 -42 -41 -40 -39 -38 +37 -36 -35 -34 -33 -32 +31 -30 -29 +28 -27 +26 +25 -24 -23 -22 +21 +20 +19 +18 +17 -16 -15 -14 -13 +12 +11 -10 -9 -8 -7 +6 +5 -4 -3 +2)
